# Generic

In [1]:
import hashlib, base64
def sha256(msg):
    hasher = hashlib.sha256()
    hasher.update(msg.encode())
    return hasher.hexdigest()
def sha1(msg):
    hasher = hashlib.sha1()
    hasher.update(msg.encode())
    return hasher.hexdigest()
def bytes_to_int(b):
    return int.from_bytes(b, 'big')
def int_to_bytes(i):
    s = hex(i)[2:].upper()
    if len(s) % 2 == 1:
        s = '0' + s
    return base64.b16decode(s)

# Crypto

In [2]:
# First, get some primitives down :

import gensafeprime

def primegen(small=True, big=False):
    """Generate a random prime."""
    if small:
        return gensafeprime.generate(32)
    if big:
        return gensafeprime.generate(1024)
    return gensafeprime.generate(128)

def egcd(r, u, v, rp, up, vp):
    """Extended Euclid algorithm for GCD
    Given a, b, find GCD(a,b) and x,y such that :
        a.x b+ b.y = GCD(a,b)     (Bézout coefficients)
    """
    if r == 0:
        return up, vp
    else:
        return egcd(rp - (rp // r)*r, up - (rp // r)*u, vp - (rp // r)*v, r, u, v)
    
    
def invmod(a, N):
    """Calculate the inverse modulo"""
    x, _ = egcd(a, 1, 0, N, 0, 1)
    return x % N

assert (invmod(15, 37) * 15) % 37 == 1

In [3]:
import base64

def str_to_int(s):
    return int(base64.b16encode(s), 16)
def int_to_str(i):
    s = hex(i)[2:].upper()
    if len(s) % 2 == 1:
        s = '0' + s # base16 wants even length strings, adding a 0 in front does not change the value. Perfect.
    return base64.b16decode(s)
assert int_to_str(str_to_int((b"Hello, world !"))) == b'Hello, world !'

class RSA:
    def __init__(self, small=True, big=False, block_length=None):
        self.p = primegen(small=small, big=big)
        self.q = primegen(small=small, big=big)
        self.n = self.p * self.q
        self.et = (self.p - 1) * (self.q - 1)
        self.e = 3
        self.d = invmod(self.e, self.et)
        if block_length is None:
            self.block_length = 3 + (not small)*9
        else:
            self.block_length = block_length
    
    @property
    def public(self):
        return self.e, self.n
    
    @property
    def private(self):
        return self.d, self.n
    
    def encrypt_block(self, s, i2s = True):
        # Encrypt one block
        if isinstance(s, bytes):
            m = str_to_int(s)
            if m ** 3 < self.n:
                print('Unsafe encryption.') # If the cubed message does not wrap around, it's not much use...
            if m > self.n:
                raise Exception("Message not encryptable. Please use blocks.")
            if i2s:
                return int_to_str(pow(m, self.e, self.n))
            else:
                return pow(m, self.e, self.n)
        else:
            m = s
            return pow(m, self.e, self.n)
    
    def decrypt_block(self, s, i2s = True):
        if isinstance(s, bytes):
            c = str_to_int(s)
            if c > self.n:
                raise Exception("Message not encryptable. Please use blocks.")
            if i2s:
                return int_to_str(pow(c, self.d, self.n))
            else:
                return pow(c, self.d, self.n)
        else:
            c = s
            return pow(c, self.d, self.n)
    
    def encrypt(self, s):
        acc = []
        for i in range(0, len(s), self.block_length):
            acc.append(self.encrypt_block(s[i:i+self.block_length]))
        return acc
        
    
    def decrypt(self, s):
        return b"".join(self.decrypt_block(b) for b in s)
        
    
r = RSA(small=False)
assert r.decrypt(r.encrypt(b"Vanilla Ice Baby")) == b'Vanilla Ice Baby'

Unsafe encryption.


# Cryptopals Notebook

## Set 6

### Exercice 41

Implement unpadded message recovery oracle

In [4]:
# Setting up legitimate side of things
class Server:
    def __init__(self):
        self.rsa = RSA(small=False)
        self.memory = set()
    
    def encrypt(self, s):
        return self.rsa.encrypt(s)
    
    def decrypt(self, s):
        if b"".join(s) in self.memory:
            raise Exception("Message already decrypted") # Why bother with hashes
        else:
            self.memory.add(b"".join(s))
            return self.rsa.decrypt(s)

# Alice is the server
alice = Server()

# Bob is the JS client
original_message = b'{ "time": 62365314455, "social": "555-555-555" }'
bob = {'message': alice.encrypt(original_message) }

# Bob asks for the decrypted message
bob['clear'] = alice.decrypt(bob['message'])
assert bob['clear'] == original_message

In [5]:
# Eve gets a hold of the ciphertext !
eve = {'message': bob['message']}

# Ensure that the server won't decrypt it again...
try:
    alice.decrypt(eve['message'])
    assert False
except:
    assert True
    
# Now modify message so that is different but still the same.
(e, n), s = alice.rsa.public, 4 # Totally random, I swear.
c = [str_to_int(block) for block in eve['message']] # Go back to ints for math
cp = [(pow(s, e, n) * block) % n for block in c]
assert c != cp
new_message = alice.decrypt([int_to_str(block) for block in cp])
assert new_message != original_message # We got a jumbled mess here

In [6]:
# Now Eve will want to recover original_message from new_message :
blocks = [str_to_int(new_message[i:i+12]) for i in range(0, len(new_message), 12)]
new_blocks = [(block * invmod(s, n)) % n for block in blocks]
decrypted_message = b"".join(int_to_str(block) for block in new_blocks)

# This is due to the fact that we add 0's to ensure int_to_str. Not that important.
# We could just make a smarter way to convert str to ints. Meh.
decrypted_message = b"".join(bytes([b]) for b in decrypted_message if b != 0)
assert decrypted_message == original_message

### Exercice 42

Bleichenbacher's e=3 RSA Attack.

In [7]:
sender = RSA(small=False, big=True)

In [8]:
def sign(sender, text):
    """Simple implementation of RSA signing"""
    d, n = sender.private
    _hash = base64.b16decode(sha1(text).upper())
    asn1 = b'\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14' # ASN.1 encoding for SHA1
    block = b'\x00\x01' + b'\xff' * (128 - len(_hash) - len(asn1) - 3) + b'\x00' + asn1 + _hash # Pad out to 128 bytes
    assert len(block) == 128 # 1024 bit blocks
    return pow(bytes_to_int(block), d, n)

def check_signature(sender, text, signature):
    """Flawed implementation of RSA signature verification"""
    e, n = sender.public
    _hash = base64.b16decode(sha1(text).upper())
    asn1 = b'\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14' # ASN.1 encoding for SHA1
    clearint = pow(signature, e, n)
    clearblock = b'\x00' + int_to_bytes(clearint) # Initial null byte is lost during integer conversion
    if not (clearblock[0:3] == b'\x00\x01\xff'):
        print('Starting bytes not ok')
        return False # Check only the "important" bytes
    clearblock = clearblock[3:]
    while clearblock[0] == 255:
        clearblock = clearblock[1:] # Consume 0xFF bytes
    return clearblock.startswith(b'\x00' + asn1 + _hash) # Check important bytes
    # So we did NOT check that the padding was the entire length
    # and we did NOT check the remaining block was equal to the ans1 + hash part.
    
# Check base case
assert check_signature(sender, "Hello, world !", sign(sender, "Hello, world !"))

This is all inspired from this submission : https://www.ietf.org/mail-archive/web/openpgp/current/msg00999.html

Let's define D and N :
$$\begin{align*}
D
&= 0x00\ ASN.1\ HASH \\
&= 2^{288} - N
\end{align*}$$

And now build the signature, using the least amount of padding possible (to have the largest possible margin) :
$$\begin{align*}
signature
&= 0x00\ 0x01\ 0xFF\ 0x00\ ASN.1\ HASH\ GARBAGE\ (1024\ bit\ key)\\
&= 2^{1009} - 2^{1000} + D.2^{712} + garbage\\
&= 2^{1009} - 2^{1000} + 2^{1000} - N.2^{712} + garbage\\
&= 2^{1009} - N.2^{712} + garbage\\
\end{align*}$$

Sadly, given that 2^1009 is not a perfect cube, we cannot make a nice formula like in the Hal Finney writeup. We still have a good base. Let's crunch some numbers to find a perfect cube that would fit our description. WolframAlpha reveals that the cube root of our baseline (2^1009 - N\*2^712) is around 1.7\*10^101. Let's use that as a starting value (note: we could have found this value by using the same binary search used to find the perfect root).

In [9]:
# Now, we want to forge a signature
e, n = sender.public
_hash = base64.b16decode(sha1('Hello, world !').upper())
asn1 = b'\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14' # ASN.1 encoding for SHA1
D = bytes_to_int(b'\x00' + asn1 + _hash)
N = 2**288 - D
target = 2**1009 - N * (2**712)
base = 176254032823638680266052181223788664320230109282117832543013801901130780612858290399771331518246797307

In [10]:
# We need a perfect cube that is slightly larger than our target (less than 2^712 difference, should be doable)
def is_good_root(n):
    if (n**3 - target) < 0:
        return -1 # Too small
    if (n**3 - target) > 2**712:
        return 1  # Too big
    return 0 # Perfect.

low = base
high = base*2
mid = (high - low) // 2 + low
is_ok = is_good_root(mid)
while is_ok != 0:
    if is_ok < 0:
        low = mid
    elif is_ok > 0:
        high = mid
    mid = (high - low) // 2 + low
    is_ok = is_good_root(mid)
root = mid
    
print('Found a good root. (root^3 - target) is %s and (2^712 - (root^3 - target)) is %s' % (mid**3 - target, 2**712 - (mid**3 - target)))

Found a good root. (root^3 - target) is 16127599852352072733114186255968266580760613075092633068618640212423478497054982267802592076314564076913940744080994100546246971644854173908828995096951797527509070032754050928806580131027491716868344335397286586176 and (2^712 - (root^3 - target)) is 5417916800390065152544908304308740433333578757271087059886318366546048406945860779574986056278438219311781496353911879206280622851898686714542534203638306433907526124188058145386474621266694133074872823712473929920


In [11]:
# Now for the test !
assert check_signature(sender, "Hello, world !", root)

### Exercice 43

